In [5]:
!pip install pecab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 10.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 9.2 MB/s eta 0:00:00
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646664 sha256=d18904fd3ce02b37eed290ca1b7e43791d1711add730e2049d96cb0a606d62fb
  Stored in directory: /home/fritzprix/.cache/pip/wheels/5c/6f/b4/ab61b8863d7d8b1409def8ae31adcaa089fa91b8d022ec309d
Successfully built pecab
  Attempting uninstall: emoji
    Found existing installation: emoji 2.9.0
    Uninstalling emoji-2.9.0:
      Successfully uninstalled emoji-2.9.0


In [9]:
from pecab import PeCab

pecab = PeCab()

pecab.morphs()

['안녕', '씨발', '새끼야', '?']

In [ ]:
from tokenizers.pre_tokenizers import PreTokenizer
from pecab import PeCab

class PeCabPreTokenizer(PreTokenizer):

    def __init__(self) -> None:
        super().__init__()
        self.pecab = PeCab()

    def pre_tokenize_str(self, sequence):
        return self.pecab.morphs(sequence)
    
    def pre_tokenize(self, pretok):
        return super().pre_tokenize(pretok)

In [23]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import ByteLevel, Sequence as PreSequence
from tokenizers.normalizers import BertNormalizer, Sequence as NormSequence, Nmt



special_tokens = [
    "<|startoftext|>",
    "<|endoftext|>",
    "<|pad|>",
    "<|cls|>",
    "<|unk|>",
    "<|instr|>",
    *[f"<|resv_{i}|>" for i in range(50)]
]

print(special_tokens)


tokenizer = Tokenizer(BPE(unk_token='<|unk|>'))

tokenizer.normalizer = NormSequence([BertNormalizer(clean_text=True, strip_accents=True), Nmt()])
tokenizer.pre_tokenizer = PreSequence([ByteLevel()])

trainer = BpeTrainer(vocab_size=40000, min_frequency=2, show_progress=True, special_tokens=special_tokens)




['<|startoftext|>', '<|endoftext|>', '<|pad|>', '<|cls|>', '<|unk|>', '<|instr|>', '<|resv_0|>', '<|resv_1|>', '<|resv_2|>', '<|resv_3|>', '<|resv_4|>', '<|resv_5|>', '<|resv_6|>', '<|resv_7|>', '<|resv_8|>', '<|resv_9|>', '<|resv_10|>', '<|resv_11|>', '<|resv_12|>', '<|resv_13|>', '<|resv_14|>', '<|resv_15|>', '<|resv_16|>', '<|resv_17|>', '<|resv_18|>', '<|resv_19|>', '<|resv_20|>', '<|resv_21|>', '<|resv_22|>', '<|resv_23|>', '<|resv_24|>', '<|resv_25|>', '<|resv_26|>', '<|resv_27|>', '<|resv_28|>', '<|resv_29|>', '<|resv_30|>', '<|resv_31|>', '<|resv_32|>', '<|resv_33|>', '<|resv_34|>', '<|resv_35|>', '<|resv_36|>', '<|resv_37|>', '<|resv_38|>', '<|resv_39|>', '<|resv_40|>', '<|resv_41|>', '<|resv_42|>', '<|resv_43|>', '<|resv_44|>', '<|resv_45|>', '<|resv_46|>', '<|resv_47|>', '<|resv_48|>', '<|resv_49|>']


In [24]:
from datasets import Dataset
from typing import List

class SuccessCaseGenerator:
    def __init__(self, datasets: List[Dataset], transform=None) -> None:
        self.datasets = datasets
        self.transform = transform

    def __call__(self, *args: Any, **kwds: Any) -> Any:
        for ds in self.datasets:
            for seq in ds:
                assert len(seq["text"]) != 0
                if self.transform:
                    seq = self.transform(seq)
                yield seq


In [22]:
from datasets import load_dataset, Dataset
subsets = [
    '20231201.ko',
    '20231201.en',
]



dataset = Dataset.from_generator(data_generator([load_dataset('wikimedia/wikisource', subset, split='train', cache_dir='cache') for subset in subsets]))
dataset.save_to_disk('cache/aggregated')

Resolving data files: 100%|██████████| 19/19 [00:00<00:00, 234387.58it/s]


TypeError: cannot pickle 'generator' object

In [25]:
from tqdm import tqdm
from datasets import load_from_disk

dataset = load_from_disk('cache/aggregated')

def batch_iterator(batch_size=1000):
    for i in range(0, len(dataset), batch_size):
        yield dataset.select(range(i, i + batch_size))['text']



tokenizer.train_from_iterator(iterator=batch_iterator(), trainer=trainer, length=len(dataset))